In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


### <b>문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현</b>
<b>문제 설명</b><br>
: 이 문제는 LangChain의 Tool Calling 기능을 학습하기 위한 기초 단계입니다. 카페 메뉴 정보를 제공하는 AI 어시스턴트를 구현하면서 다양한 데이터 소스(로컬 DB, 웹, 위키피디아)에서 정보를 검색하는 방법을 익힙니다.

<b>Tool 정의 방법 이해</b>: @tool 데코레이터를 사용한 사용자 정의 도구 생성<br>
<b>벡터 DB 구축</b>: 텍스트 데이터를 임베딩하여 검색 가능한 형태로 저장<br>
<b>다중 도구 활용</b>: 서로 다른 용도의 도구들을 하나의 LLM에 연결<br>
<b>기본 체인 구성</b>: 도구 호출 결과를 처리하는 간단한 워크플로우 구현<br>

<b>요구사항</b>
1. 카페 메뉴 데이터 파일 생성 및 벡터 DB 구축
2. 3개의 도구를 정의하고 LLM에 바인딩
3. 간단한 도구 호출 체인 구현체인 구조: 사용자 질문 → LLM (도구 선택) → 도구 실행 → 결과 종합 → 최종 답변
4. 테스트 질문 처리

In [6]:
import re
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
from pprint import pprint

#############################################
# 1. 카페 메뉴 데이터 파일 생성 및 벡터 DB 구축
#############################################
loader = TextLoader("../data/cafe_menu_data.txt", encoding="UTF-8")
documents = loader.load()

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = [] # [Document, Document]
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(type(doc))
    pprint(vars(doc))
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("./db/cafe_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 4},
)

총 10개의 메뉴 항목이 처리되었습니다.
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '아메리카노',
              'menu_number': 1,
              'source': '../data/cafe_menu_data.txt'},
 'page_content': '1. 아메리카노\n'
                 '   • 가격: ₩4,500\n'
                 '   • 주요 원료: 에스프레소, 뜨거운 물\n'
                 '   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 '
                 '잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.',
 'type': 'Document'}

메뉴 번호: 1
메뉴 이름: 아메리카노
내용:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 ...
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '카페라떼',
              'menu_number': 2,
              'source': '../data/cafe_menu_data.txt'},
 'page_content': '2. 카페라떼\n'
                 '   • 가격: ₩5,500\n'
                 '   • 주요 원료: 에스프레소, 스팀 밀크\n'
                 '   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 '
    

In [7]:
from langchain_core.tools import tool
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from textwrap import dedent
from langchain.vectorstores import FAISS

#############################################
# 2. 3개의 도구 정의 및 LLM 바인딩
#############################################

# a) tavily_search_func
@tool 
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

# b) wiki_summary
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

class WikiSearchSchema(BaseModel):
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

wiki_runnable = RunnableLambda(search_wiki)
wiki_summary = wiki_runnable.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)

# c) db_search_cafe_func 래핑 및 RunnableTool 생성

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# RunnableLambda로 감싸기 (입력 dict 형태로 받도록 래핑)
def db_search_cafe_runnable_func(input_data: dict) -> List[Document]:
    query = input_data.get("query", "")
    return db_search_cafe_func(query)

class DBSchema(BaseModel):
    query: str = Field(..., description="검색할 메뉴 이름")

db_search_cafe_runnable = RunnableLambda(db_search_cafe_runnable_func)
db_search_cafe_tool = db_search_cafe_runnable.as_tool(
    name="db_search_cafe_func",
    description="Search cafe menu DB securely.",
    args_schema=DBSchema
)

# LLM 및 도구 리스트에 추가
llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [
    tavily_search_func,
    wiki_summary,
    db_search_cafe_tool,   # 래핑된 도구 사용
]
llm_with_tools = llm.bind_tools(tools=tools)


C:\Users\혜영\AppData\Local\Temp\ipykernel_21644\2844436930.py:47: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_summary = wiki_runnable.as_tool(


In [8]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import ToolMessage

#############################################
# 3. 간단한 도구 호출 체인 구현체인 구조:
#############################################
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}. When the user asks about prices, always first use the tool named 'db_search_cafe_func' to search the cafe database."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools
print(type(llm_chain))

@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print(type(ai_msg))
    print("ai_msg: \n", ai_msg)
    print("-"*100)
    print("TOOL CALLS 구조 확인")
    pprint(ai_msg.tool_calls)

    tool_msgs = []

    for call in ai_msg.tool_calls:
        name = call['name']
        args = call['args']

        if name == "db_search_cafe_func":
            # 여기서 db_search_cafe_tool.invoke 사용
            result_docs = db_search_cafe_tool.invoke(args)
            content = "\n\n".join([doc.page_content for doc in result_docs])
            tool_msgs.append(ToolMessage(tool_call_id=call['id'], content=content))

        elif name == "tavily_search_func":
            # tavily_search_func는 그냥 함수 호출
            result = tavily_search_func(args['query'])
            tool_msgs.append(ToolMessage(tool_call_id=call['id'], content=result))

        elif name == "wiki_summary":
            result_docs = wiki_summary.invoke(args)
            content = "\n\n".join([doc.page_content for doc in result_docs])
            tool_msgs.append(ToolMessage(tool_call_id=call['id'], content=content))

    print(type(tool_msgs))
    print("tool_msgs: \n", tool_msgs)
    
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)
# 체인 실행
query = "아메리카노 가격은 얼마인가요?"
response = web_search_chain.invoke(query)

# 응답 출력 
pprint(response.content)

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_core.messages.ai.AIMessage'>
ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_wsQ6oN5v5gPFSLfXaxwiyTEV', 'function': {'arguments': '{"query":"아메리카노"}', 'name': 'db_search_cafe_func'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 234, 'total_tokens': 255, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BjpLo35h0iw9yb58irECc4v5C01OD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--6475a13d-f1cf-4429-b29e-d5e3de30aebb-0' tool_calls=[{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_wsQ6oN5v5gPFSLfXaxwiyTEV', 'type': 'tool_ca

In [9]:
#############################################
# 4. 테스트 질문 처리
#############################################
from pprint import pprint

# 4단계: 테스트 질문 처리

def test_query_handling(query: str):
    print(f"질문: {query}\n{'='*80}")
    response = web_search_chain.invoke(query)
    pprint(response.content)
    print("\n" + "="*80 + "\n")

# 실제 테스트 실행
test_query_handling("아메리카노 가격은 얼마인가요?")
test_query_handling("커피의 역사에 대해서 알려줘")
test_query_handling("이 가게 메뉴에 카페라떼가 있나요?")
test_query_handling("아메리카노의 가격과 특징은 무엇인가요?")


질문: 아메리카노 가격은 얼마인가요?
<class 'langchain_core.messages.ai.AIMessage'>
ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_0dRzO1lYCMtBV9Phg71r63XZ', 'function': {'arguments': '{"query":"아메리카노"}', 'name': 'db_search_cafe_func'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 234, 'total_tokens': 255, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BjpLs8jcLrZnCWQNnv7qhyEjO62Db', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--d8dfd5b8-8924-4cdc-8f6d-144dfaf4f07f-0' tool_calls=[{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_0dRzO1lYCMtBV9Phg71r63XZ', 'type': 'tool_call'}] usage_metadata={'input_tokens'

### <b>문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트</b>
<b>문제 설명</b><br>
: 문제 1의 기본 체인을 발전시켜, Few-shot 프롬프팅 기법을 적용한 고급 AI 어시스턴트를 구현합니다. 이를 통해 AI가 언제 어떤 도구를 사용해야 하는지 더 정확하게 판단할 수 있도록 합니다.

<b>Few-shot 프롬프팅 이해</b>: 예제를 통한 AI 행동 패턴 학습<br>
<b>고급 프롬프트 엔지니어링</b>: 시스템 메시지와 예제의 효과적 조합<br>
<b>복합 질문 처리</b>: 여러 도구를 순차적으로 사용하는 워크플로우<br>
<b>멀티모달 정보 통합</b>: 서로 다른 소스의 정보를 하나로 종합<br>

<b>요구사항</b>
1. Few-shot 예제를 포함한 프롬프트 템플릿 작성
2. 각 도구의 용도를 명확히 구분하는 시스템 메시지 작성
3. 도구 실행 결과를 종합하여 최종 답변을 생성하는 체인 구현
4. 복합 질문 처리 테스트

In [10]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    MessagesPlaceholder
)
from typing import List, Dict, Any

#############################################
# 1. Few-shot 예제를 포함한 프롬프트 템플릿 작성
#############################################

# Few-shot 예제 구성
few_shot_examples = [
      {
        "input": "아메리카노 정보와 커피 역사를 알려주세요",
        "tool_calls": [
            {"name": "db_search_cafe_func", "args": {"query": "아메리카노"}},
            {"name": "wiki_summary", "args": {"query": "커피 역사"}}
        ],
        "output": "아메리카노는 에스프레소에 뜨거운 물을 추가한 음료로... (종합 정보)"
    }
]
# Few-shot 프롬프트 템플릿 생성
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# Few-shot 프롬프트 구성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=few_shot_examples,
    input_variables=["input", "output"]  # 명시적 변수 지정
)

#####################################################
# 2. 각 도구의 용도를 명확히 구분하는 시스템 메시지 작성
#####################################################

# 강화된 시스템 메시지
system_message = f"""당신은 카페 메뉴 정보와 음료 관련 지식을 제공하는 AI 어시스턴트입니다. 오늘 날짜는 {today}입니다. 

도구 사용 가이드라인:
1. db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명 등)
2. wiki_summary: 음료의 역사, 문화적 배경 등 일반 지식
3. tavily_search_func: 최신 트렌드, 뉴스 등 실시간 정보

도구 선택 원칙:
- 메뉴 관련 질문 → db_search_cafe_func 우선 사용
- 역사/문화 질문 → wiki_summary 사용
- 최신 정보 필요 → tavily_search_func 사용
- 복합 질문 → 필요한 도구 모두 사용
- 정보 출처 명시 필수

응답 구조:
1. 메뉴 정보: 가격, 설명 포함
2. 일반 지식: 간결한 요약
3. 최신 정보: 출처 명시
4. 추천: 근거 제시"""

full_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    few_shot_prompt,
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)  # 선택적 사용
])
##########################################################
# 3. 도구 실행 결과를 종합하여 최종 답변을 생성하는 체인 구현
##########################################################

def format_tool_messages(messages: List[Dict[str, Any]]) -> List[ToolMessage]:
    """도구 호출 결과를 ToolMessage로 변환"""
    tool_msgs = []
    for msg in messages:
        if msg["role"] == "tool":
            tool_msgs.append(
                ToolMessage(
                    content=msg["content"],
                    tool_call_id=msg["tool_call_id"]
                )
            )
    return tool_msgs

@chain
def advanced_search_chain(user_input: str, config: RunnableConfig):
    # 초기 호출 (예제 변수 포함)
    initial_input = {
        "input": user_input,
        "output": "",  # 빈 값으로 초기화
        "agent_scratchpad": []  # 초기 빈 리스트
    }
    
    # 첫 번째 LLM 호출
    ai_msg = (full_prompt | llm_with_tools).invoke(initial_input, config=config)
    
    # 도구 호출 처리
    tool_msgs = []
    if hasattr(ai_msg, 'tool_calls') and ai_msg.tool_calls:
        for call in ai_msg.tool_calls:
            try:
                tool_name = call["name"]
                args = call["args"]
                
                # 도구 실행
                if tool_name == "db_search_cafe_func":
                    result = db_search_cafe_tool.invoke(args)
                    content = "\n".join([doc.page_content for doc in result])
                elif tool_name == "wiki_summary":
                    result = wiki_summary.invoke(args)
                    content = "\n".join([doc.page_content for doc in result])
                elif tool_name == "tavily_search_func":
                    content = tavily_search_func.invoke(args["query"])
                
                tool_msgs.append(ToolMessage(
                    content=content,
                    tool_call_id=call["id"]
                ))
            except Exception as e:
                print(f"도구 실행 오류: {e}")
                continue
    
    # 응답 생성
    final_input = {
        "input": user_input,
        "output": "",  # 이전 응답 없음
        "agent_scratchpad": [ai_msg] + tool_msgs
    }
    response = (full_prompt | llm).invoke(final_input, config=config)
    return response

#############################################
# 4. 복합 질문 처리 테스트
#############################################

def run_test_case(query: str):
    print(f"테스트 질문: {query}")
    print("=" * 80)
    try:
        response = advanced_search_chain.invoke(query)
        print(response.content)
    except Exception as e:
        print(f"오류 발생: {type(e).__name__}: {e}")
    print("=" * 80 + "\n")

# 단일 질문 테스트
run_test_case("카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요.")

# 다양한 테스트 케이스
test_cases = [
    "플랫 화이트에 대해 설명해주시고, 이 음료의 기원도 알려주세요",
    "요즘 인기 있는 차 음료를 추천해주시고, 관련 트렌드도 알려주세요",
    "에스프레소 머신의 역사와 우리 카페에서 판매하는 에스프레소 메뉴를 비교 설명해주세요"
]

for query in test_cases:
    run_test_case(query)

테스트 질문: 카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요.


C:\Users\혜영\AppData\Local\Temp\ipykernel_21644\2844436930.py:20: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=2)


카페라떼는 에스프레소에 뜨거운 우유를 첨가한 음료로, 이탈리아어로 '커피와 우유'를 의미하는 "caffè e latte"에서 유래되었습니다. 17세기 유럽에서 시작되었으며, 현대적인 형태의 라떼는 20세기에 등장했습니다. [출처: 위키백과]

카페라떼와 잘 어울리는 디저트로는 크루아상, 블루베리 머핀, 바나나 브레드 등이 있습니다. 크루아상의 부드럽고 담백한 맛이 라떼의 크리미한 풍미와 잘 어울리며, 블루베리 머핀의 상큼함과 바나나 브레드의 달콤함이 라떼의 맛을 더욱 돋보이게 합니다. [출처: 오하시스 블로그, 농업인 신문]

테스트 질문: 플랫 화이트에 대해 설명해주시고, 이 음료의 기원도 알려주세요
### 플랫 화이트 정보
- **가격**: ₩4,500
- **주요 원료**: 에스프레소, 마이크로폼
- **설명**: 플랫 화이트는 에스프레소와 마이크로폼으로 구성된 커피 음료입니다. 카페 라떼와 비슷하지만, 부피가 작고 마이크로폼이 적습니다. 따라서 우유 대비 커피 비율이 높아 에스프레소 맛이 지배적입니다.

### 플랫 화이트의 기원
플랫 화이트는 1980년대 중순 오스트레일리아 시드니의 Moors Espresso Bar에서 처음 기록되었습니다. Alan Preston은 1985년 이 음료를 자신의 메뉴에 추가했습니다. 플랫 화이트는 카페 라떼와 비교하여 우유 대비 커피 비율이 더 높아 강한 커피 맛을 느낄 수 있습니다. 

출처: 위키백과 요약

테스트 질문: 요즘 인기 있는 차 음료를 추천해주시고, 관련 트렌드도 알려주세요
요즘 인기 있는 차 음료로는 "녹차 라떼"와 "콜드브루"가 있습니다.

1. **녹차 라떼**
   - 가격: ₩5,800
   - 주요 원료: 말차 파우더, 스팀 밀크, 설탕
   - 설명: 고급 말차 파우더와 부드러운 스팀 밀크로 만든 건강한 음료입니다. 녹차의 은은한 쓴맛과 우유의 부드러움이 조화를 이루며, 항산화 성분이 풍부합니다. 달콤함 조절이 가능합니다.

2. **콜드브루**
   - 가격: ₩5,000
   - 주요 원